In [ ]:
import lib._util.visualplot as vp

In [ ]:
import pandas as pd
import numpy as np

# Scikit-Learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# Plotly
import plotly.graph_objects as go

# Time measurement
import time
from datetime import timedelta

# Sound notification
import winsound

# Useful Functions

In [ ]:
SOURCE_PATH_DATA = 'resources/data/'
OUT_PATH_GRAPH = 'resources/output/data_preparation/graph/'
OUT_PATH_FILE = 'resources/output/data_preparation/file/'

def time_taken(seconds):
    print(f'\nTime Taken: {str(timedelta(seconds=seconds))}')
    winsound.Beep(frequency=1000, duration=100)
    winsound.Beep(frequency=1500, duration=50)

# Data Preparation (Raw Data)

### Data Loading

In [ ]:
def load_data(currency_pair, periods):
    df_list = []
    for period in periods:
        source_file = f'resources/data/DAT_ASCII_{currency_pair}_T_{period}.csv'
        df_chunks   = pd.read_csv(source_file, sep=',',
                                  header=None, names=['datetime', 'bid', 'ask', 'vol'],
                                  usecols=['datetime', 'bid', 'ask'],
                                  parse_dates=['datetime'],
                                  date_parser=lambda x: pd.to_datetime(x, format='%Y%m%d %H%M%S%f'),
                                  chunksize=50_000)

        df = pd.concat(df_chunks)
        df_list.append(df)

    return pd.concat(df_list)

In [ ]:
EXEC_START = time.time()

currency_pair = 'EURUSD'
periods       = [f'2019{str(x+1).zfill(2)}' for x in range(6)]

timeseries_df = load_data(currency_pair, periods)

EXEC_END = time.time()
time_taken(EXEC_END - EXEC_START)

In [ ]:
vp.fast_stat(timeseries_df)

### Feature Engineering
- Timeframe aggregation
- Calculate oscillators
  1. RSI
  2. Stochastic RSI
  3. Fast & Slow Stochastic
  4. MACD

In [ ]:
def aggregate(df, rule):
    # Resampling
    bid_df = df.set_index('datetime')['bid'].resample(rule).ohlc().reset_index()
    ask_df = df.set_index('datetime')['ask'].resample(rule).ohlc().reset_index()

    bid_df.dropna(inplace=True)
    ask_df.dropna(inplace=True)

    bid_df.reset_index(drop=True, inplace=True)
    ask_df.reset_index(drop=True, inplace=True)
    
    new_df = pd.DataFrame({
        'datetime': bid_df['datetime'],

        'open_bid': bid_df['open'],
        'high_bid': bid_df['high'],
        'low_bid': bid_df['low'],
        'bid': bid_df['close'],

        'open_ask': ask_df['open'],
        'high_ask': ask_df['high'],
        'low_ask': ask_df['low'],
        'ask': ask_df['close']
    })
    new_df['datetime'] = new_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Oscillator
    n_timestep = 14
    new_df = calc_roc(new_df, n_timestep)
    new_df = calc_rsi(new_df, n_timestep)
    new_df = calc_stochastic_rsi(new_df, n_timestep)
    new_df = calc_stochastic(new_df, n_timestep, slow_timestep=3)
    new_df = calc_macd(new_df)
    
    # Decimal rounding
    for column in [x for x in new_df.columns if x != 'datetime']:
        new_df[column] = np.round(new_df[column], 5)
    
    return new_df

In [ ]:
# Reference:
# - https://www.youtube.com/watch?v=Gbnhbr4HhG0
# - https://www.marketvolume.com/technicalanalysis/roc.asp
def calc_roc(df, n_timestep):
    new_df = df.copy()
    
    for row in new_df[n_timestep -1:].itertuples():
        bid_nperiod = new_df.at[row.Index - (n_timestep -1), 'bid']
        ask_nperiod = new_df.at[row.Index - (n_timestep -1), 'ask']
        
        new_df.at[row.Index, 'bid_roc'] = (row.bid - bid_nperiod) / bid_nperiod * 100
        new_df.at[row.Index, 'ask_roc'] = (row.ask - ask_nperiod) / ask_nperiod * 100
    
    return new_df

In [ ]:
# Reference:
# - https://www.youtube.com/watch?v=WZbOeFsSirM
def calc_rsi(df, n_timestep):
    new_df = df.copy()
    tmp_df = df.copy()
    
    tmp_df['bid_movement'] = tmp_df['bid'].diff()
    tmp_df['ask_movement'] = tmp_df['ask'].diff()

    tmp_df['bid_upward_movement']   = np.where(tmp_df['bid_movement'] > 0, tmp_df['bid_movement'], 0)
    tmp_df['bid_downward_movement'] = np.where(tmp_df['bid_movement'] < 0, np.abs(tmp_df['bid_movement']), 0)

    tmp_df['ask_upward_movement']   = np.where(tmp_df['ask_movement'] > 0, tmp_df['ask_movement'], 0)
    tmp_df['ask_downward_movement'] = np.where(tmp_df['ask_movement'] < 0, np.abs(tmp_df['ask_movement']), 0)

    tmp_df.at[n_timestep -1, 'bid_avg_upward_movement']   = tmp_df[['bid_upward_movement']][:n_timestep].values.mean()
    tmp_df.at[n_timestep -1, 'bid_avg_downward_movement'] = tmp_df[['bid_downward_movement']][:n_timestep].values.mean()

    tmp_df.at[n_timestep -1, 'ask_avg_upward_movement']   = tmp_df[['ask_upward_movement']][:n_timestep].values.mean()
    tmp_df.at[n_timestep -1, 'ask_avg_downward_movement'] = tmp_df[['ask_downward_movement']][:n_timestep].values.mean()

    tmp_df = tmp_df[n_timestep -1:].copy()
    tmp_df.reset_index(inplace=True, drop=True)

    for row in tmp_df[1:].itertuples():
        tmp_df.at[row.Index, 'bid_avg_upward_movement']   = (tmp_df.at[row.Index -1, 'bid_avg_upward_movement'] * (n_timestep -1) + row.bid_upward_movement) / n_timestep
        tmp_df.at[row.Index, 'bid_avg_downward_movement'] = (tmp_df.at[row.Index -1, 'bid_avg_downward_movement'] * (n_timestep -1) + row.bid_downward_movement) / n_timestep

        tmp_df.at[row.Index, 'ask_avg_upward_movement']   = (tmp_df.at[row.Index -1, 'ask_avg_upward_movement'] * (n_timestep -1) + row.ask_upward_movement) / n_timestep
        tmp_df.at[row.Index, 'ask_avg_downward_movement'] = (tmp_df.at[row.Index -1, 'ask_avg_downward_movement'] * (n_timestep -1) + row.ask_downward_movement) / n_timestep

    tmp_df['bid_relative_strength'] = tmp_df['bid_avg_upward_movement'] / tmp_df['bid_avg_downward_movement']
    tmp_df['ask_relative_strength'] = tmp_df['ask_avg_upward_movement'] / tmp_df['ask_avg_downward_movement']

    tmp_df['bid_rsi'] = 100 - (100 / (tmp_df['bid_relative_strength'] + 1))
    tmp_df['ask_rsi'] = 100 - (100 / (tmp_df['ask_relative_strength'] + 1))

    tmp_df = tmp_df[['datetime', 'bid_rsi', 'ask_rsi']]
    
    return new_df.merge(tmp_df, on='datetime', how='left')

In [ ]:
# Reference:
# - https://www.youtube.com/watch?v=cGDUQCCELMo
# - https://www1.oanda.com/forex-trading/learn/trading-tools-strategies/stochastic
def calc_stochastic_rsi(df, n_timestep):
    new_df = df.copy()
    tmp_df = new_df[new_df['bid_rsi'].isna() == False].copy()
    tmp_df.reset_index(inplace=True, drop=True)
    
    tmp_df['roll_high_bid_rsi'] = tmp_df['bid_rsi'].rolling(n_timestep).max()
    tmp_df['roll_high_ask_rsi'] = tmp_df['ask_rsi'].rolling(n_timestep).max()
    tmp_df['roll_low_bid_rsi']  = tmp_df['bid_rsi'].rolling(n_timestep).min()
    tmp_df['roll_low_ask_rsi']  = tmp_df['ask_rsi'].rolling(n_timestep).min()
    
    tmp_df.dropna().reset_index(inplace=True, drop=True)
    tmp_df['bid_stochastic_rsi'] = (tmp_df['bid_rsi'] - tmp_df['roll_low_bid_rsi']) / (tmp_df['roll_high_bid_rsi'] - tmp_df['roll_low_bid_rsi']) * 100
    tmp_df['ask_stochastic_rsi'] = (tmp_df['ask_rsi'] - tmp_df['roll_low_ask_rsi']) / (tmp_df['roll_high_ask_rsi'] - tmp_df['roll_low_ask_rsi']) * 100
    
    tmp_df = tmp_df[['datetime', 'bid_stochastic_rsi', 'ask_stochastic_rsi']]
    
    return new_df.merge(tmp_df, on='datetime', how='left')

In [ ]:
# Reference:
# - https://www.youtube.com/watch?v=1UaPhm-TIkw
# - https://www.investopedia.com/ask/answers/05/062405.asp
# - https://www1.oanda.com/forex-trading/learn/trading-tools-strategies/stochastic
def calc_stochastic(df, n_timestep, slow_timestep):
    new_df = df.copy()
    tmp_df = df.copy()
    
    tmp_df['roll_high_bid'] = tmp_df['bid'].rolling(n_timestep).max()
    tmp_df['roll_high_ask'] = tmp_df['ask'].rolling(n_timestep).max()
    tmp_df['roll_low_bid']  = tmp_df['bid'].rolling(n_timestep).min()
    tmp_df['roll_low_ask']  = tmp_df['ask'].rolling(n_timestep).min()
    
    tmp_df = tmp_df[tmp_df['roll_high_bid'].isna() == False].copy()
    tmp_df.reset_index(inplace=True, drop=True)
    
    tmp_df['bid-rlb'] = tmp_df['bid'] - tmp_df['roll_low_bid']
    tmp_df['rhb-rlb'] = tmp_df['roll_high_bid'] - tmp_df['roll_low_bid']
    tmp_df['ask-rla'] = tmp_df['ask'] - tmp_df['roll_low_ask']
    tmp_df['rha-rla'] = tmp_df['roll_high_ask'] - tmp_df['roll_low_ask']
    
    tmp_df['bid_fast_stochastic'] = tmp_df['bid-rlb'] / tmp_df['rhb-rlb'] * 100
    tmp_df['ask_fast_stochastic'] = tmp_df['ask-rla'] / tmp_df['rha-rla'] * 100
    
    tmp_df['bid_slow_stochastic'] = tmp_df['bid-rlb'].rolling(slow_timestep).sum() / tmp_df['rhb-rlb'].rolling(slow_timestep).sum() * 100
    tmp_df['ask_slow_stochastic'] = tmp_df['ask-rla'].rolling(slow_timestep).sum() / tmp_df['rha-rla'].rolling(slow_timestep).sum() * 100
    
    tmp_df = tmp_df[['datetime', 'bid_fast_stochastic', 'ask_fast_stochastic', 'bid_slow_stochastic', 'ask_slow_stochastic']]
    
    return new_df.merge(tmp_df, on='datetime', how='left')

In [ ]:
# Reference:
# - https://www.youtube.com/watch?v=so0NMh67ySw
# - https://www.investopedia.com/terms/m/macd.asp
def calc_macd(df):
    new_df = df.copy()
    
    fast_ema_timestep = 12
    slow_ema_timestep = 26
    signal_timestep   = 9
    
    fast_ema_factor = 2 / (fast_ema_timestep +1)
    slow_ema_factor = 2 / (slow_ema_timestep +1)
    signal_factor   = 2 / (signal_timestep +1)
    
    # Fast EMA
    tmp_df = df.copy()
    tmp_df.at[fast_ema_timestep -1, 'bid_fast_ema'] = tmp_df[['bid']][:fast_ema_timestep].values.mean()
    tmp_df.at[fast_ema_timestep -1, 'ask_fast_ema'] = tmp_df[['ask']][:fast_ema_timestep].values.mean()
    
    tmp_df = tmp_df[fast_ema_timestep -1:].copy()
    tmp_df.reset_index(inplace=True, drop=True)

    for row in tmp_df[1:].itertuples():
        prev_bfema = tmp_df.at[row.Index -1, 'bid_fast_ema']
        prev_afema = tmp_df.at[row.Index -1, 'ask_fast_ema']
        
        tmp_df.at[row.Index, 'bid_fast_ema'] = ((row.bid - tmp_df.at[row.Index -1, 'bid_fast_ema']) * fast_ema_factor) + prev_bfema
        tmp_df.at[row.Index, 'ask_fast_ema'] = ((row.ask - tmp_df.at[row.Index -1, 'ask_fast_ema']) * fast_ema_factor) + prev_afema
    
    tmp_df = tmp_df[['datetime', 'bid_fast_ema', 'ask_fast_ema']]
    new_df = new_df.merge(tmp_df, on='datetime', how='left')
    
    # Slow EMA
    tmp_df = df.copy()
    tmp_df.at[slow_ema_timestep -1, 'bid_slow_ema'] = tmp_df[['bid']][:slow_ema_timestep].values.mean()
    tmp_df.at[slow_ema_timestep -1, 'ask_slow_ema'] = tmp_df[['ask']][:slow_ema_timestep].values.mean()
    
    tmp_df = tmp_df[slow_ema_timestep -1:].copy()
    tmp_df.reset_index(inplace=True, drop=True)

    for row in tmp_df[1:].itertuples():
        prev_bsema = tmp_df.at[row.Index -1, 'bid_slow_ema']
        prev_asema = tmp_df.at[row.Index -1, 'ask_slow_ema']
        
        tmp_df.at[row.Index, 'bid_slow_ema'] = ((row.bid - tmp_df.at[row.Index -1, 'bid_slow_ema']) * slow_ema_factor) + prev_bsema
        tmp_df.at[row.Index, 'ask_slow_ema'] = ((row.ask - tmp_df.at[row.Index -1, 'ask_slow_ema']) * slow_ema_factor) + prev_asema
    
    tmp_df = tmp_df[['datetime', 'bid_slow_ema', 'ask_slow_ema']]
    new_df = new_df.merge(tmp_df, on='datetime', how='left')
    
    # EMA differences
    new_df['bid_ema_diff'] = new_df['bid_fast_ema'] - new_df['bid_slow_ema']
    new_df['ask_ema_diff'] = new_df['ask_fast_ema'] - new_df['ask_slow_ema']
    
    # Signal
    tmp_df = new_df[new_df['bid_ema_diff'].isna() == False].copy()
    tmp_df.reset_index(inplace=True, drop=True)
    
    tmp_df.at[signal_timestep -1, 'bid_macd_signal'] = tmp_df[['bid_ema_diff']][:signal_timestep].values.mean()
    tmp_df.at[signal_timestep -1, 'ask_macd_signal'] = tmp_df[['ask_ema_diff']][:signal_timestep].values.mean()
    
    tmp_df = tmp_df[signal_timestep -1:].copy()
    tmp_df.reset_index(inplace=True, drop=True)

    for row in tmp_df[1:].itertuples():
        prev_bsignal = tmp_df.at[row.Index -1, 'bid_macd_signal']
        prev_asignal = tmp_df.at[row.Index -1, 'ask_macd_signal']
        
        tmp_df.at[row.Index, 'bid_macd_signal'] = ((row.bid_ema_diff - tmp_df.at[row.Index -1, 'bid_macd_signal']) * signal_factor) + prev_bsignal
        tmp_df.at[row.Index, 'ask_macd_signal'] = ((row.ask_ema_diff - tmp_df.at[row.Index -1, 'ask_macd_signal']) * signal_factor) + prev_asignal
    
    tmp_df = tmp_df[['datetime', 'bid_macd_signal', 'ask_macd_signal']]
    new_df = new_df.merge(tmp_df, on='datetime', how='left')
    
    # Histogram
    new_df['bid_macd_histogram'] = new_df['bid_ema_diff'] - new_df['bid_macd_signal']
    new_df['ask_macd_histogram'] = new_df['ask_ema_diff'] - new_df['ask_macd_signal']
    
    return new_df.drop(columns=['bid_ema_diff', 'ask_ema_diff'])

In [ ]:
# Aggregation (Daily)
day_df = aggregate(timeseries_df, rule='1D')
vp.fast_stat(day_df)

In [ ]:
# Aggregation (Hourly)
hour_df = aggregate(timeseries_df, rule='1H')
vp.fast_stat(hour_df)

In [ ]:
# Aggregation (Minute)
min_df = aggregate(timeseries_df, rule='1Min')
vp.fast_stat(min_df)

In [ ]:
# Drop N/A
day_df.dropna(inplace=True)
hour_df.dropna(inplace=True)
min_df.dropna(inplace=True)

In [ ]:
# Export
EXEC_START = time.time()

day_df.to_csv(f'{OUT_PATH_FILE}/DAT_ASCII_{currency_pair}_Day_{periods[0]}-{periods[-1]}.csv', index=False)
hour_df.to_csv(f'{OUT_PATH_FILE}/DAT_ASCII_{currency_pair}_Hour_{periods[0]}-{periods[-1]}.csv', index=False)
min_df.to_csv(f'{OUT_PATH_FILE}/DAT_ASCII_{currency_pair}_Min_{periods[0]}-{periods[-1]}.csv', index=False)

EXEC_END = time.time()
time_taken(EXEC_END - EXEC_START)

# Data Preparation (Transformed Data)

### Data Loading

In [ ]:
currency_pair = 'EURUSD'
filename      = f'DAT_ASCII_{currency_pair}_Day_201901-201906.csv'
# filename      = f'DAT_ASCII_{currency_pair}_Hour_201901-201906.csv'
# filename      = f'DAT_ASCII_{currency_pair}_Min_201901-201906.csv'

source_file = f'{OUT_PATH_FILE}{filename}'
df_chunks   = pd.read_csv(source_file, sep=',',
                          parse_dates=['datetime'],
                          date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'),
                          chunksize=50_000)
timeseries_df = pd.concat(df_chunks)

### Data Normalization

In [ ]:
def skewed_handling(df, columns):
    new_df = df.copy()
    
    for column in columns:
        new_df[f'{column}_log'] = np.log1p(df[column])
        
    return new_df

def standard_scaler(df, columns):
    new_df = df.copy()
    
    scaled_values = StandardScaler().fit_transform(df[columns])
    new_df = pd.concat([
        new_df,
        pd.DataFrame(scaled_values, columns=[f'{x}_norm' for x in columns])
    ], axis=1)
    
    return new_df

In [ ]:
# Log transform skewed data
# No skewed data on oscillator values

In [ ]:
# Normalize data
columns = [x for x in timeseries_df.columns if any([y for y in ['roc', 'rsi', 'stochastic', 'ema', 'macd'] if y in x])]
timeseries_df = standard_scaler(timeseries_df, columns)
timeseries_df.drop(columns=columns, inplace=True)

### Dimensionality Reduction

In [ ]:
def pca_evaluation(values):
    shape = values.shape
    print(shape)
    
    pca = PCA(n_components=shape[1])
    pca.fit(values)
    
    # Reference: https://www.appliedaicourse.com/lecture/11/applied-machine-learning-online-course/2896/pca-for-dimensionality-reduction-not-visualization/0/free-videos
    # Evaluate by the variance, and try to preserve variance as high as 90%
    expvar_percentages    = pca.explained_variance_ / np.sum(pca.explained_variance_)
    cumexpvar_percentages = np.cumsum(expvar_percentages)
    
    return cumexpvar_percentages

def pcaeval_plot(cumexpvar_percentages):
    data = []
    data.append(go.Scattergl(
        x = [x for x in range(1, len(cumexpvar_percentages) +1)],
        y = cumexpvar_percentages,
        mode = 'lines+markers'
    ))
    
    vp.plot_graph(data, 'PCA Evaluation',
                  xlabel='N Features', ylabel='Cumulative Explained Variance',
                  out_path=OUT_PATH_GRAPH)

def pca_reduction(df, columns, n_component):
    values = df[columns]
    pca    = PCA(n_components=n_component)
    reduced_values = pca.fit_transform(values)
    
    new_df = pd.concat([
        df,
        pd.DataFrame(reduced_values, columns=[f'pca_{x}' for x in range(1, n_component +1)])
    ], axis=1)
    
    return new_df

In [ ]:
# PCA evaluation
columns = [x for x in timeseries_df.columns if any([y for y in ['roc', 'rsi', 'stochastic', 'ema', 'macd'] if y in x])]
values  = timeseries_df[columns].values
cumexpvar_percentages = pca_evaluation(values)

pcaeval_plot(cumexpvar_percentages)

In [ ]:
# PCA reduction
n_component   = 4
columns       = [x for x in timeseries_df.columns if any([y for y in ['roc', 'rsi', 'stochastic', 'ema', 'macd'] if y in x])]
timeseries_df = pca_reduction(timeseries_df, columns, n_component)

### Data Normalization

In [ ]:
def minmax_scaler(df, columns):
    new_df = df.copy()
    
    scaled_values = MinMaxScaler(feature_range=(0, 1)).fit_transform(df[columns])
    new_df = pd.concat([
        new_df,
        pd.DataFrame(scaled_values, columns=[f'{x}_norm' for x in columns])
    ], axis=1)
    
    return new_df

In [ ]:
columns       = [x for x in timeseries_df.columns if 'pca_' in x]
timeseries_df = minmax_scaler(timeseries_df, columns)

In [ ]:
# Export
EXEC_START = time.time()

# Decimal rounding
for column in [x for x in timeseries_df.columns if x != 'datetime']:
    timeseries_df[column] = np.round(timeseries_df[column], 5)
timeseries_df.to_csv(f'{OUT_PATH_FILE}/DAT_ASCII_{currency_pair}_Normalized.csv', index=False)

EXEC_END = time.time()
time_taken(EXEC_END - EXEC_START)